# 1. Подготовка

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('toxic_comments.csv')

In [3]:
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [9]:
%%time
df['text'] = df['text'].str.lower() 


df1 = df.sample(frac=0.1).reset_index(drop=True)

#df1 = df

Wall time: 253 ms


<b> лематизируем данные и очистим их от стопслов

In [10]:
#!pip install -U textblob --user

#!python -m textblob.download_corpora

In [11]:
#!pip install -U textblob --user

In [12]:
from textblob import TextBlob, Word
from nltk.corpus import stopwords
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Killdone\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [13]:
#Используем текстблог с постегом для привода предложения в изначальную форму
%time
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

#sentence = "The striped bats are hanging on their feet for best"
#lemmatize_with_postag(sentence)
#> 'The striped bat be hang on their foot for best'
df1['text_lemm'] = df1['text'].apply(lambda x: lemmatize_with_postag(x))
#corpus = list(df['text'])
#lemmatize_with_postag(corpus[0])
#> 'The striped bat be hang on their foot for best'

Wall time: 0 ns


На 80к данных
CPU times: user 8min 56s, sys: 7.54 s, total: 9min 3s
Wall time: 9min 8s

In [14]:
df1

,text,toxic,text_lemm
0,new sock of the editor of all things wikipedia...,0,new sock of the editor of all thing wikipedia ...
1,deletion discussion about monopoly (star wars)...,0,deletion discussion about monopoly star war he...
2,"~no one has mentioned gambling in montana, so ...",0,~no one have mention gamble in montana so i do...
3,though i cannot comment on the distribution la...,0,though i can not comment on the distribution l...
4,", about a street elephant named pang dao",0,about a street elephant name pang dao
...,...,...,...
15952,"""\n\nan added remark in my article that will p...",0,an added remark in my article that will please...
15953,"""::::i fear they might have intellectual prope...",0,:i fear they might have intellectual property ...
15954,funny how you constantly make up lies. i have ...,0,funny how you constantly make up lie i have no...
15955,beerstraaten or beerstraten?\nin the dutch lan...,0,beerstraaten or beerstraten in the dutch langu...


In [15]:
import re

In [16]:
#Очистим текст и разделим соеденим слов, разделяя их пробелом
df1['text_lemm'] = df1['text'].apply(lambda x: re.sub(r'[^a-zA-Z\']', ' ', x))
df1['text_lemm'] = df1['text_lemm'].apply(lambda x: ' '.join(x.split()))

corpus = df1['text_lemm']
df1

,text,toxic,text_lemm
0,new sock of the editor of all things wikipedia...,0,new sock of the editor of all things wikipedia...
1,deletion discussion about monopoly (star wars)...,0,deletion discussion about monopoly star wars h...
2,"~no one has mentioned gambling in montana, so ...",0,no one has mentioned gambling in montana so i ...
3,though i cannot comment on the distribution la...,0,though i cannot comment on the distribution la...
4,", about a street elephant named pang dao",0,about a street elephant named pang dao
...,...,...,...
15952,"""\n\nan added remark in my article that will p...",0,an added remark in my article that will please...
15953,"""::::i fear they might have intellectual prope...",0,i fear they might have intellectual property i...
15954,funny how you constantly make up lies. i have ...,0,funny how you constantly make up lies i have n...
15955,beerstraaten or beerstraten?\nin the dutch lan...,0,beerstraaten or beerstraten in the dutch langu...


# Способ без BERT

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords as nltk_stopwords
#from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.model_selection import train_test_split


In [52]:
corpus = df1['text_lemm'].values
target = df1['toxic']
 
corpus_train, corpus_test, target_train, target_test = train_test_split(corpus, target, test_size=0.2)

In [53]:
#Очистим текст от стопслов и превратим в признаки
nltk.download('stopwords')
 
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
count_tf_idf.fit(corpus_train)
 
features_train = count_tf_idf.transform(corpus_train)
features_test = count_tf_idf.transform(corpus_test)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Killdone\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# BERT

In [ ]:
#!pip install -U tensorflow --user

In [ ]:
#!pip install torch===1.5.1 torchvision===0.6.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
#!pip install transformers

In [ ]:
#! pip install tensorflow-gpu

In [17]:
import torch
import transformers
from tqdm import notebook
import tensorflow as tf

In [18]:
df2 = df1

In [69]:
df2 = df1.sample(2000).reset_index(drop=True) #Возьмем небольшой кусок датасета 

In [70]:
df2['text_lemm']

0       i agree leave the links alone already but insh...
1       assistance at cultural appropriation hi i'd ap...
2       i guess not since it is stated by neutral hist...
3       barnstar the excellent user page award to date...
4       complain about your disruptive behavior here h...
                              ...                        
1995    hi qqq please note that pages in the user name...
1996    hi danimations i have some suggestions firstly...
1997    decausa your pissing me off first of all i hav...
1998    you're always the victim aren't you jzg i'll b...
1999    impact on plastics recycling does rubber tough...
Name: text_lemm, Length: 2000, dtype: object

In [71]:
#random.seed(400)
#data_bert=data.loc[random.sample(range(len(data)),1200)]

tokenizer = transformers.BertTokenizer( vocab_file='vocab.txt')
tokenized = df2['text_lemm'].apply(lambda x: tokenizer.encode(x[:512], add_special_tokens=True))
n = max(map(len, tokenized))

config = transformers.BertConfig.from_json_file(
    'bert-base-uncased-config.json')
model = transformers.BertModel.from_pretrained(
    'bert-base-uncased-pytorch_model.bin', config=config)

batch_size = 100
embeddings = []
for i in notebook.tqdm(range(tokenized.shape[0] // batch_size)):
    
        padded = np.array([i + [0]*(n - len(i)) for i in tokenized.values[batch_size*i:batch_size*(i+1)]])
        attention_mask = np.where(padded != 0, 1, 0)
        
        batch = torch.LongTensor(padded) 
        attention_mask_batch = torch.LongTensor(attention_mask)
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())



In [72]:
features = np.concatenate(embeddings)
features.shape

(2000, 768)

In [73]:
features_train_b,features_test_b, target_train_b,target_test_b=train_test_split(features, df2['toxic'], test_size=0.5)

In [74]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# 2. Обучение

<b>LogisticRegression

In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
model_lr = LogisticRegression(class_weight = 'balanced', solver='liblinear', random_state=12345)
 
model_lr.fit(features_train, target_train) 
predict = model_lr.predict(features_test)
print(f1_score(target_test, predict))

0.7098591549295774


<b>LogisticRegression BERT

In [77]:
model_lrb = LogisticRegression(class_weight = 'balanced', solver='liblinear', random_state=12345)
model_lrb.fit(features_train_b, target_train_b)
predictions= model_lrb.predict(features_test_b)
print('bert f1_score:', f1_score(target_test_b, predictions))

bert f1_score: 0.24999999999999994


In [78]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from  sklearn.ensemble import RandomForestClassifier
#from catboost import CatBoostClassifer

<b>RandomForestClassifier

In [79]:
model = RandomForestClassifier(max_depth = 15, n_estimators = 40, class_weight = 'balanced', random_state=12345)
model.fit(features_train, target_train)
predict = model.predict(features_test)
print(f1_score(target_test, predict))

0.44187963726298435


In [ ]:
#model = RandomForestClassifier()
parameters = {'max_depth'         : [10, 15, 30, 50],
                  'n_estimators' : [10, 20, 40, 60],
                  
                 }
grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 2, n_jobs=-1, scoring='f1')
grid.fit(features_train, target_train)    
grid.predict(features_test)
         
    
print("\n The best estimator across ALL searched params:\n",
              grid.best_estimator_)
    
print("\n The best score across ALL searched params:\n",
              grid.best_score_)
    
print("\n The best parameters across ALL searched params:\n",
              grid.best_params_)
print("\n ========================================================")

    

In [ ]:
print(grid.predict(features_test))

In [ ]:
print(f1_score(target_test, grid.predict(features_test)))

In [60]:
#!pip install catboost

<b>RandomForestClassifier BERT

In [80]:
model = RandomForestClassifier(max_depth = 15, n_estimators = 40, class_weight = 'balanced', random_state=12345)
model.fit(features_train_b, target_train_b)
predict = model.predict(features_test_b)
print(f1_score(target_test_b, predict))

I:\Python\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0


<b>CatBoostClassifier

In [65]:
from sklearn.utils import shuffle
from catboost import CatBoostClassifier

In [66]:
model = CatBoostClassifier(depth =10, learning_rate=1, iterations =30, random_state=12345)
model.fit(features_train, target_train)
predict = model.predict(features_test)
print(f1_score(target_test, predict))

0:	learn: 0.2806386	total: 4.54s	remaining: 2m 11s
1:	learn: 0.2408580	total: 8.74s	remaining: 2m 2s


KeyboardInterrupt: 

In [ ]:
#model = CatBoostClassifier()
parameters = {'depth'         : [6,8,10],
                  'learning_rate' : [0.01, 0.05, 0.1],
                  'iterations'    : [30, 50, 100]
                 }
grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 2, n_jobs=-1, scoring='f1')
grid.fit(features_train, target_train)    
y_pred_c = grid.predict(features_test)
   

    # Results from Grid Search
print("\n========================================================")
print(" Results from Grid Search " )
print("========================================================")    
    
print("\n The best estimator across ALL searched params:\n",
          grid.best_estimator_)
    
print("\n The best score across ALL searched params:\n",
          grid.best_score_)
    
print("\n The best parameters across ALL searched params:\n",
          grid.best_params_)
print("\n ========================================================")


<b>CatBoostClassifier BERT

In [81]:
model = CatBoostClassifier(depth =10, learning_rate=1, iterations =10, random_state=12345)
model.fit(features_train_b, target_train_b)
predict = model.predict(features_test_b)
print(f1_score(target_test_b, predict))

0:	learn: 0.2848270	total: 2.36s	remaining: 21.3s
1:	learn: 0.2037210	total: 4.7s	remaining: 18.8s
2:	learn: 0.1440010	total: 7.1s	remaining: 16.6s
3:	learn: 0.1087328	total: 9.43s	remaining: 14.1s
4:	learn: 0.0612010	total: 12s	remaining: 12s
5:	learn: 0.0471180	total: 14.4s	remaining: 9.58s
6:	learn: 0.0385294	total: 16.7s	remaining: 7.16s
7:	learn: 0.0352107	total: 19s	remaining: 4.76s
8:	learn: 0.0328804	total: 21.4s	remaining: 2.38s
9:	learn: 0.0263911	total: 23.8s	remaining: 0us
0.14814814814814817


# 3. Выводы

По хорошему Линейная Регрессия выдает лучший результат. Она быстро обучается и не тербует настройки параметров. Вероятно ей проще всего работать с текстами(из известных мне моделей). Кэт буст так же справляется с задачей, но крайне долго. 



# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны